In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
import sys
sys.path.append("..")
import pickle as pkl
import hist
from analyzer.datasets import SampleManager
from analyzer.core import AnalysisResult
import math
import torch
import gpytorch
from torch.masked import masked_tensor, as_masked_tensor
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from analyzer.plotting.plots_1d import drawAs1DHist
from analyzer.plotting.plots_2d import drawAs2DHist, addTitles2D
from analyzer.plotting.plottables import PlotObject
from analyzer.plotting.mplstyles import loadStyles
import fitting.regression as regression
import fitting.plot_tools as fplt
import fitting.high_level as fhl
from gpytorch.kernels import ScaleKernel as SK
from gpytorch.kernels import RBFKernel as RBF
import linear_operator
import fitting.models as models
torch.set_default_dtype(torch.float64)
plt.close("auto")

%matplotlib inline

In [ ]:
res = AnalysisResult.fromFile("../results/data_control.pkl")

sample_manager = SampleManager()
sample_manager.loadSamplesFromDirectory("../datasets")
hists = res.getMergedHistograms(sample_manager)
sig_res = AnalysisResult.fromFile("../results/everything.pkl")
sig_res.results["signal_312_1500_600"].histograms["h_njet"] 
sighists = sig_res.getMergedHistograms(sample_manager)

In [ ]:
bkg_name="CR0b_Data2018"
complete_hist = hists["ratio_m14_vs_m24"]
narrowed = complete_hist
narrowed =  complete_hist[...,hist.loc(1000):hist.loc(3000),hist.loc(0.35):hist.loc(1.0)]
narrowed = narrowed[...,::hist.rebin(1),::hist.rebin(1)]
qcd_hist = narrowed[bkg_name,...]
sig_hist = sighists["ratio_m14_vs_m24"]["signal_312_1500_900",hist.loc(1000):hist.loc(3000),hist.loc(0.35):hist.loc(1)]
qcd_hist = narrowed[bkg_name,...] 
true_sum = qcd_hist.sum()

In [ ]:
fig,ax= plt.subplots(1,2,figsize=(10,3))
_ = drawAs2DHist(ax[0], PlotObject.fromHist(qcd_hist))
_ = drawAs2DHist(ax[1], PlotObject.fromHist(sig_hist))

In [ ]:
import fitting.regression as fr
window = [(1300, 1750), (400, 700)]
rebin = 1
mask_f = fr.rectMasker(window)
#mask_f = fr.ellipseMasker(torch.tensor([1500.0,600.0]), 200.0,150.0)
#mask_f = fr.ellipseMasker(torch.tensor([1400.0,0.6]), 150.0,0.05)
mask_f = fr.ellipseMasker(torch.tensor([2000.0,0.6]), 150.0,0.05)
mask_f = fr.ellipseMasker(torch.tensor([1200.0,0.5]), 100.0,0.05)
#mask_f = fr.ellipseMasker(torch.tensor([1800.0,0.6]), 50.0,0.02)
#mask_f = fr.ellipseMasker(torch.tensor([1500.0,1400.0]), 200.0,150.0)
#mask_f = fr.ellipseMasker(torch.tensor([1450.0,0.59]), 150.0,0.05)
#mask_f=None

#mask_f = fr.ellipseMasker(torch.tensor([1200.0,0.6]), 200.0,0.05)
exclude_less=0.001
#mask_f = None
train_data = regression.makeRegressionData(qcd_hist[hist.rebin(rebin),hist.rebin(rebin)], mask_f, exclude_less=exclude_less)
test_data, m = regression.makeRegressionData(qcd_hist[hist.rebin(rebin),hist.rebin(rebin)], None, exclude_less=exclude_less, get_mask=True)
train_transform = regression.getNormalizationTransform(train_data)
normalized_train_data = train_transform.transform(train_data)
normalized_test_data = train_transform.transform(test_data)

fig,ax = plt.subplots(2,2,figsize=(10,10),layout="tight")
fplt.simpleGrid(ax[0][0], train_data.E, train_data.X, train_data.Y)
mask = regression.getBlindedMask(test_data.X, test_data.Y, test_data.Y, test_data.V, mask_f)
squares = fplt.makeSquares(test_data.X[mask], test_data.E)
points = fplt.getPolyFromSquares(squares)
_=drawAs2DHist(ax[0][1], PlotObject.fromHist(sig_hist))
poly = matplotlib.patches.Polygon(points, edgecolor="red", fill=False)
ax[0][1].add_patch(poly)
signal_data = regression.makeRegressionData(sig_hist[hist.rebin(rebin),hist.rebin(rebin)])
signal_data = regression.DataValues(signal_data.X[m], signal_data.Y[m], signal_data.V[m], signal_data.E)
fplt.simpleGrid(ax[1][0], signal_data.E, signal_data.X, signal_data.Y)
fplt.simpleGrid(ax[1][1], normalized_train_data.E, normalized_train_data.X, normalized_train_data.V)

In [ ]:
use_cuda = True 
if torch.cuda.is_available() and use_cuda:   
    print("Loading train to gpu")
    train=regression.sendToGpu(normalized_train_data)
    #train=regression.sendToGpu(train_data)
    print("Loaded train to gpu")
else:
    train = normalized_train_data
linear=gpytorch.kernels.LinearKernel(ard_num_dims=2,num_dimensions=2)
rq=SK(gpytorch.kernels.RQKernel(ard_num_dims=2))

smk = gpytorch.kernels.SpectralMixtureKernel(ard_num_dims=2, num_mixtures=8)
gsmk=models.GeneralSpectralMixture(num_mixtures=1, ard_num_dims=2)
#peak_smk.initialize_from_data(train.X, train.Y)

peaked_rbf=SK(models.PeakedRBF(ard_num_dims=2))
peaked_grbf=SK(models.PeakedGRBF(ard_num_dims=2))

NNRBF = models.wrapNN("NNRBFKernel", gpytorch.kernels.RBFKernel)
NNRQ = models.wrapNN("NNRQKernel", gpytorch.kernels.RQKernel)
NNGRBF = models.wrapNN("NNRBFKernel", models.GeneralRBF)




grq=SK(models.GeneralRQ(ard_num_dims=2))
grbf = SK(models.GeneralRBF(ard_num_dims=2))
gmatern = SK(models.GeneralMatern(ard_num_dims=2))
poly = SK(gpytorch.kernels.PiecewisePolynomialKernel(ard_num_dims=2))
lk = SK(gpytorch.kernels.LinearKernel(ard_num_dims=2,num_dimensions=2))
rbf = SK(gpytorch.kernels.RBFKernel(ard_num_dims=2))
matern = SK(gpytorch.kernels.MaternKernel(ard_num_dims=2))


NNSMKernel = models.wrapNN("NNSMKernel", gpytorch.kernels.SpectralMixtureKernel)
nnrbf = SK(NNRBF(odim=2,layer_sizes=(256,64,32,16,4,)))
#nnrbf = SK(NNRBF(odim=2,layer_sizes=(32,32,16)))
#nnrbf = SK(NNRBF(odim=2,layer_sizes=(256,128,64,32)))
nnsmk = NNSMKernel(idim=2, odim=2, num_mixtures=2)


#shape = (256, 64,32)
#shape = (256,128,16)
#shape = (1024,1024,128)
shape = (32,16,8)



nngrbf = SK(NNGRBF(odim=2,layer_sizes=shape))
nnrbf = SK(NNRBF(odim=2,layer_sizes=shape))

if use_cuda and False:
    smk = smk.cuda()
    nnsmk = nnsmk.cuda()
    smk.initialize_from_data(train.X, train.Y)
    nnsmk.initialize_from_data(nnsmk.feature_extractor(train.X),train.Y)

#model,likelihood = regression.createModel(train, kernel=grbf)

grid_size = gpytorch.utils.grid.choose_grid_size(train.X,1.0)
grid_size = 10
print(grid_size)
grid_rbf = SK(gpytorch.kernels.GridInterpolationKernel(
    gpytorch.kernels.RBFKernel(ard_num_dims=2),grid_size=grid_size,num_dims=2))
likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise=train.V, learn_additional_noise=False)
model=models.InducingPointModel(train.X,train.Y, 
                                likelihood, 
                                kernel=grbf, 
                                inducing=train.X[::2])
#model,likelihood = regression.createModel(train, kernel=grbf)
if torch.cuda.is_available() and use_cuda:       
    print("Loading model to gpu")
    model = model.cuda()
    likelihood = likelihood.cuda()
    print("Loaded model to gpu")
print(model)

#model = model.cuda()
def pyro_model(x, y):
    with gpytorch.settings.fast_computations(False, False, False):
        sampled_model = model.pyro_sample_from_prior()
        output = sampled_model.likelihood(sampled_model(x))
        print(output.mean)
        print(y)
        pyro.sample("obs", output, obs=y)
    return y
#print(train.X)
#nuts_kernel = NUTS(pyro_model, adapt_step_size=True)
#mcmc_run = MCMC(nuts_kernel, num_samples=10, warmup_steps=10)
#mcmc_run.run(train.X, train.Y)

model,likelihood = regression.optimizeHyperparams(model,likelihood, train, bar=False, iterations=200, lr=0.1)
print("DONE")
if torch.cuda.is_available() and use_cuda:   
    print("Loading model from gpu")
    model = model.cpu()
    likelihood = likelihood.cpu()
    print("Loaded model from gpu")

In [ ]:
#if mask_f:
  #  mask = regression.getBlindedMask(pred.X, pred.Y, test_data.Y, test_data.V, mask_f)

p = regression.getPrediction(model, likelihood, normalized_test_data)
e = linear_operator.utils.cholesky.psd_safe_cholesky(p.covariance_matrix)
cm = e @ e.T
pred_dist=p
print(torch.allclose(cm,p.covariance_matrix))
#pred_dist = gpytorch.distributions.MultivariateNormal(p.mean,cm)
#pred_dist = likelihood(pred_dist)
pred = train_transform.iTransform(regression.DataValues(normalized_test_data.X, pred_dist.mean, pred_dist.variance.detach(), normalized_test_data.E))

if mask_f:
    mask = regression.getBlindedMask(pred.X, pred.Y, test_data.Y, test_data.V, mask_f)
    bpred_mean = pred.Y[mask]
    obs_mean = test_data.Y[mask]
    obs_var = test_data.V[mask]
    chi2 = torch.sum((obs_mean - bpred_mean)**2 / obs_var) / torch.count_nonzero(mask)
    avg_pull = torch.sum(torch.abs((obs_mean - bpred_mean)) / torch.sqrt(obs_var)) / torch.count_nonzero(mask)
    print(f"Chi^2/bins = {chi2}")
    print(f"Avg Abs pull = {avg_pull}")
else:
    mask=None
if avg_pull < 5:    
    p = fhl.makeDiagnosticPlots(pred, test_data, train_data, qcd_hist, mask)

In [ ]:
params = list(model.named_parameters_and_constraints())
for name,param,constraint in params:
    if constraint:
        real_param = constraint.transform(param)
    else:
        real_param = param
    print(f"{name.replace('raw_',''):{max(len(x[0]) for x in params)+4}} {real_param.detach().numpy().round(3)}" )

In [ ]:
import jax
e,e2=jnp.linalg.eigh(cmnp)
e2

In [ ]:
import numpyro
import jax.numpy as jnp
import numpyro.distributions as dist
from numpyro.handlers import condition, seed, substitute, trace


V = linear_operator.utils.cholesky.psd_safe_cholesky(pred_dist.covariance_matrix).numpy()
cm = V @ V.T
evals,evecs = jnp.linalg.eigh(cm)
evals = evals[::-1]
evecs = evecs[::-1]
eva = evals[:50]
eve = evecs[:50].T

evam = jnp.sqrt(jnp.diag(eva))
#print(jnp.diag(eva).shape)
#print(eve.shape)
X= jnp.matmul(eve, evam)
print(V.shape)
print(V)

mu = pred_dist.mean.numpy()
sd = signal_data.Y.numpy()
print(sd)

def pyro_model(observation=None):        
    r= numpyro.sample("rate",dist.Uniform(0,20))  
    #with numpyro.plate("background_variations", eva.shape[0]): 
    #    background = numpyro.sample("background_estimation_raw", dist.Normal(0,1))
    with numpyro.plate("background_variations", V.shape[0]): 
        b = numpyro.sample("background_estimation_raw", dist.Normal(0,1))
    background = mu + V @ b        
    obs_hist = (r * sd) +  transform.iTransformData(background)
    with numpyro.plate("bins", mu.shape[0]):         
        return numpyro.sample("observed", dist.Normal(jnp.clip(obs_hist,1),jnp.sqrt(jnp.clip(obs_hist,1))), obs=observation) 
        #return numpyro.sample("observed", dist.Poisson(jnp.ceil(jnp.clip(obs_hist,1))), obs=observation)

observation = transform.iTransformData(mu) + 10 * sd
observation = jnp.ceil(jnp.clip(obs,1))

#conditioned = condition(pyro_model, {"observed": obs})
#tm = reparam(pyro_model, config={"background_estimation": LocScaleReparam(0)})
print(obs)
if False:
    fig,ax = plt.subplots()
    ax.hist([pyro_model(observation=observation)[1].cpu() for x in range(1000)])

conditioned = condition(pyro_model, {"observed" : observation})

In [ ]:
rng_key = random.PRNGKey(0)
rng_key, rng_key_ = random.split(rng_key)
nuts_kernel = NUTS(conditioned,adapt_step_size=True)
mcmc = MCMC(nuts_kernel, num_samples=1000, num_warmup=500,num_chains=1)
mcmc.run(rng_key)

In [ ]:
mcmc.print_summary()

In [ ]:
def is_outlier(points, thresh=3.5):
    if len(points.shape) == 1:
        points = points[:,None]
    median = torch.median(points, axis=0).values
    print(median)
    diff = torch.sum((points - median)**2, axis=-1)
    diff = torch.sqrt(diff)
    med_abs_deviation = torch.median(diff)
    print(med_abs_deviation)
    modified_z_score = 0.6745 * diff / med_abs_deviation
    return modified_z_score > thresh
def noOutliers(a, thresh = 3.5):
    return a[~is_outlier(a,thresh)]
    
s = mcmc.get_samples()
fig,ax = plt.subplots()

ax.hist(s['rate'])

In [ ]:
pyro.clear_param_store()

guide = pyro.infer.autoguide.AutoNormal(conditioned)
#def guide():
#    pass
num_steps = 4000
initial_lr = 0.1
gamma = 0.01  # final learning rate will be gamma * initial_lr
lrd = gamma ** (1 / num_steps)
adam = pyro.optim.ClippedAdam({'lr': initial_lr, 'lrd': lrd})
#adam = pyro.optim.Adam({"lr": 0.01})  # Consider decreasing learning rate.
elbo = pyro.infer.Trace_ELBO()
svi = pyro.infer.SVI(conditioned, guide, adam, elbo)
losses = []
for step in range(num_steps):  # Consider running for more steps.
    loss = svi.step()
    losses.append(loss)
    if step % ( num_steps // 10) == 0:
        #print(f"r = {pyro.param('r').item()}")
        print("Elbo loss: {:0.3f}".format(loss))



In [ ]:
from pyro.infer import Predictive
predictive = Predictive(conditioned, guide=guide, num_samples=1000)
p = predictive()

In [ ]:
r = p['rate'].detach().flatten()
fig,ax = plt.subplots()
ax.hist(r,bins=20)


In [ ]:
import fitting.plot_tools as fpt
import analyzer.plotting as plotting
from pathlib import Path
import importlib

importlib.reload(fpt)
dim = 1

pred_mean,_ = fitting.regression.pointsToGrid(test_data.X, pred.Y, test_data.E)
pred_var,_ = fitting.regression.pointsToGrid(test_data.X, pred.V, test_data.E)
obs_vals,_ = fitting.regression.pointsToGrid(test_data.X, test_data.Y, test_data.E)
obs_vars, filled = fitting.regression.pointsToGrid(test_data.X, test_data.V, test_data.E)

#(figpath /"slices" / f"along_{dim}").mkdir(parents=True, exist_ok=True)
for val, f, ax in fpt.createSlices(
    pred_mean.hist,
    pred_var.hist,
    obs_vals.hist,
    obs_vars.hist,
    test_data.E,
    filled,
    mask_function=mask_f,
    observed_title="CRData", slice_dim=dim, just_window=True):
    plotting.addTitles1D(ax, plotting.PlotObject.fromHist(qcd_hist[:,sum]))
    #f.savefig(figpath /"slices" / f"along_{dim}" /  (f"slice_{round(float(val),3)}".replace(".","p") + ".pdf"))
    #plt.close(f)
    

In [ ]:
from gpytorch.models.deep_gps import DeepGPLayer, DeepGP
from gpytorch.mlls import DeepApproximateMLL
from gpytorch.means import ConstantMean, LinearMean
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.variational import VariationalStrategy, CholeskyVariationalDistribution
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import ApproximateGP, GP
from gpytorch.mlls import VariationalELBO, AddedLossTerm
from gpytorch.likelihoods import GaussianLikelihood
from torch.utils.data import TensorDataset, DataLoader

if torch.cuda.is_available() and use_cuda:   
    print("Loading train to gpu")
    train=regression.sendToGpu(normalized_train_data)
    test=regression.sendToGpu(normalized_test_data)
    #train = regression.DataValues(normalized_train_data.X.cuda(), normalized_train_data.Y.cuda(), normalized_train_data.V.cuda(), None)
    print("Loaded train to gpu")
else:
    train = normalized_train_data
    test = normalized_train_data

smoke_test = True
train_x, train_y, train_v = train.X, train.Y, train.V
test_x, test_y, test_v = test.X, test.Y, test.V


train_dataset = TensorDataset(train_x, train_y, train_v)
train_loader = DataLoader(train_dataset, batch_size=100000, shuffle=True)


class ToyDeepGPHiddenLayer(DeepGPLayer):
    def __init__(self, input_dims, output_dims, num_inducing=128, mean_type='constant'):
        if output_dims is None:
            inducing_points = torch.randn(num_inducing, input_dims) + 1
            batch_shape = torch.Size([])
        else:
            inducing_points = torch.randn(output_dims, num_inducing, input_dims) + 1
            batch_shape = torch.Size([output_dims])

        variational_distribution = CholeskyVariationalDistribution(
            num_inducing_points=num_inducing,
            batch_shape=batch_shape
        )

        variational_strategy = VariationalStrategy(
            self,
            inducing_points,
            variational_distribution,
            learn_inducing_locations=True
        )

        super(ToyDeepGPHiddenLayer, self).__init__(variational_strategy, input_dims, output_dims)

        if mean_type == 'constant':
            self.mean_module = ConstantMean(batch_shape=batch_shape)
        else:
            self.mean_module = LinearMean(input_dims)
        self.covar_module = ScaleKernel(
            RBFKernel(batch_shape=batch_shape, ard_num_dims=input_dims),
            batch_shape=batch_shape, ard_num_dims=None
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

    def __call__(self, x, *other_inputs, **kwargs):
        """
        Overriding __call__ isn't strictly necessary, but it lets us add concatenation based skip connections
        easily. For example, hidden_layer2(hidden_layer1_outputs, inputs) will pass the concatenation of the first
        hidden layer's outputs and the input data to hidden_layer2.
        """
        if len(other_inputs):
            if isinstance(x, gpytorch.distributions.MultitaskMultivariateNormal):
                x = x.rsample()

            processed_inputs = [
                inp.unsqueeze(0).expand(gpytorch.settings.num_likelihood_samples.value(), *inp.shape)
                for inp in other_inputs
            ]

            x = torch.cat([x] + processed_inputs, dim=-1)
        return super().__call__(x, are_samples=bool(len(other_inputs)))

num_hidden_dims =  10

class DeepGP(DeepGP):
    def __init__(self, train_x_shape, train_vars):
        hidden_layer = ToyDeepGPHiddenLayer(
            input_dims=train_x_shape[-1],
            output_dims=num_hidden_dims,
            mean_type='linear',
        )
        
        middle_hidden = ToyDeepGPHiddenLayer(
            input_dims=num_hidden_dims,
            output_dims=num_hidden_dims,
            mean_type='linear',
        )

        last_layer = ToyDeepGPHiddenLayer(
            input_dims=middle_hidden.output_dims,
            output_dims=None,
            mean_type='constant',
        )
        
        super().__init__()
        
        self.hidden_layer = hidden_layer
        self.last_layer = last_layer
        self.middle_hidden = middle_hidden
        self.likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise=train_vars)
        #self.likelihood = gpytorch.likelihoods.GaussianLi(noise=train_vars)

    def forward(self, inputs):
        hidden_rep1 = self.hidden_layer(inputs)
        x = self.middle_hidden(hidden_rep1)
        output = self.last_layer(hidden_rep1)
        return output
    
    def predict(self, test_loader):
        with torch.no_grad():
            mus = []
            variances = []
            lls = []
            for x_batch, y_batch in test_loader:
                preds = self.likelihood(self(x_batch))
                mus.append(preds.mean)
                variances.append(preds.variance)
                lls.append(model.likelihood.log_marginal(y_batch, model(x_batch)))
        
        return torch.cat(mus, dim=-1).mean(0), torch.cat(variances, dim=-1).mean(0), torch.cat(lls, dim=-1)

model = DeepGP(train_x.shape,train_v)
if torch.cuda.is_available():
    model = model.cuda()

# this is for running the notebook in our testing framework
num_epochs =  5000
num_samples = 4

optimizer = torch.optim.Adam([{'params': model.parameters()},], lr=0.05)
mll = DeepApproximateMLL(VariationalELBO(model.likelihood, model, train_x.shape[-2]))

epochs_iter = range(num_epochs)
for i in epochs_iter:
    # Within each iteration, we will go over each minibatch of data    minibatch_iter = train_loader
    
    #for x_batch, y_batch, noise in train_loader:
    with gpytorch.settings.num_likelihood_samples(num_samples):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y, noise=train_v)
        loss.backward()
        optimizer.step()
    if i % (num_epochs // 10) == 0 :
        print(f"Epoch {i:3}: Loss {loss:0.3f}")
print("DONE")


In [ ]:
import gpytorch
import math


test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=1000000)

model.eval()
#predictive_means, predictive_variances, test_lls = model.predict(test_loader)
with torch.no_grad(): 
    p = model.likelihood(model(test_x),noise=test_v)
    predictive_means = p.mean.mean(0)
    predictive_variances = p.variance.mean(0)

if torch.cuda.is_available() and use_cuda:   
    predictive_means=predictive_means.cpu()
    predictive_variances=predictive_variances.cpu()
    #train = regression.DataValues(normalized_train_data.X.cuda(), normalized_train_data.Y.cuda(), normalized_train_data.V.cuda(), None)

pred = test_transform.iTransform(
    regression.DataValues(normalized_test_data.X, predictive_means, predictive_variances, normalized_test_data.E))

import fitting.high_level as fhl
predictive_means

p = fhl.makeDiagnosticPlots(pred, test_data, train_data, qcd_hist)

In [ ]:
import pyro

class PyroGPModel(gpytorch.models.PyroGP):
    def __init__(self, train_x, train_y, likelihood, inducing_points, kernel=None, mean=None):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            num_inducing_points=inducing_points.size(0),
        )
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, inducing_points, 
            variational_distribution, 
            learn_inducing_locations=True
        )
        super().__init__(
            variational_strategy,
            likelihood,
            num_data=train_y.numel(),
            name_prefix="simple_regression_model"
        )
        self.likelihood = likelihood
        self.mean_module = mean or gpytorch.means.ConstantMean()
        if kernel is None:
            kernel = gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.RBFKernel(ard_num_dims=2)
            )
        self.covar_module = kernel

    def forward(self, x):
        mean = self.mean_module(x)  # Returns an n_data vec
        covar = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean, covar)


class StandardApproximateGP(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points,kernel=None, mean=None):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, 
            inducing_points, 
            variational_distribution, 
            learn_inducing_locations=True
        )
        super().__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        
        self.mean_module = mean or gpytorch.means.ConstantMean()
        if kernel is None:
            kernel = gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.RBFKernel(ard_num_dims=2)
            )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
        
use_cuda = True 

if torch.cuda.is_available() and use_cuda:   
    print("Loading train to gpu")
    train=regression.sendToGpu(normalized_train_data)
    print("Loaded train to gpu")
else:
    train = normalized_train_data

NNRBF = fitting.models.wrapNN("NNRBFKernel", gpytorch.kernels.RBFKernel)
nnrbf = SK(NNRBF(odim=2,layer_sizes=(256,128,64,32)))
nnrbf = SK(NNRBF(odim=2,layer_sizes=(32,16)))

likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise=train.V)
model=StandardApproximateGP(
                 #train.X,
                 #train.Y, 
                 #likelihood, 
                 train.X[::4],
                 kernel=grbf
              )

if torch.cuda.is_available() and use_cuda:   
    print("Loading model to gpu")
    model = model.cuda()
    likelihood = likelihood.cuda()
    print("Loaded model to gpu")

def trainModel(model, likelihood, lr=0.01, num_iter = 1):
    optimizer = pyro.optim.Adam({"lr": lr})
    elbo = pyro.infer.Trace_ELBO(num_particles=128, vectorize_particles=True, retain_graph=True)
    svi = pyro.infer.SVI(model.model, model.guide, optimizer, elbo)
    model.train()
    likelihood.train()

    iterator = range(num_iter)
    for i in iterator:
        model.zero_grad()
        loss = svi.step(train.X, train.Y)
        if i % (num_iter // 10) == 0 :
            print(f"Iter {i}: Loss {loss:2}")

    return model

def trainModel2(model, likelihood,lr=0.1,num_iter = 1):
    model.train()
    likelihood.train()
    optimizer = torch.optim.Adam([
        {'params': model.parameters()},
        {'params': likelihood.parameters()},
    ], lr=lr)
    
    # Our loss object. We're using the VariationalELBO
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train.Y.size(0))
    
    
    iterator = range(num_iter)
    for i in iterator:
        optimizer.zero_grad()
        output = model(train.X)
        loss = -mll(output, train.Y)
        loss.backward()
        optimizer.step()
        
        if i % (num_iter // 10) == 0 :
            print(f"Iter {i}: Loss {loss:2}")
    
    print(f"Iter {i}: Loss {loss:2}")
    return model
    

print(model)
model = trainModel2(model,likelihood,lr=0.025,num_iter=10000)



print("DONE")
#with linear_operator.settings.max_cg_iterations(2000):
#    model,likelihood = regression.optimizeHyperparams(model,likelihood, train, bar=False, iterations=250, lr=0.05)
print("DONE")
if torch.cuda.is_available() and use_cuda:   
    print("Loading model to gpu")
    model = model.cpu()
    likelihood = likelihood.cpu()
    print("Loaded model to gpu")

In [ ]:
params = list(model.named_parameters_and_constraints())
for name,param,constraint in params:
    if constraint:
        real_param = constraint.transform(param)
    else:
        real_param = param
    print(f"{name.replace('raw_',''):{max(len(x[0]) for x in params)+4}} {real_param.detach().numpy().round(3)}" )

In [ ]:

pred_dist = regression.getPrediction(model, likelihood, normalized_test_data)
pred = test_transform.iTransform(    regression.DataValues(normalized_test_data.X, pred_dist.mean, pred_dist.variance, normalized_test_data.E))

if window:
    mask = regression.getBlindedMask(pred.X, pred.Y, test_data.Y, test_data.V, mask_f)
    bpred_mean = pred.Y[mask]
    obs_mean = test_data.Y[mask]
    obs_var = test_data.V[mask]
    chi2 = torch.sum((obs_mean - bpred_mean)**2 / obs_var) / torch.count_nonzero(mask)
    avg_pull = torch.sum(torch.abs((obs_mean - bpred_mean)) / torch.sqrt(obs_var)) / torch.count_nonzero(mask)
    print(f"Chi^2/bins = {chi2}")
    print(f"Avg Abs pull = {avg_pull}")
if avg_pull < 5:    
    p = fhl.makeDiagnosticPlots(pred, test_data, train_data, qcd_hist, mask)

In [ ]:

p = fhl.makeDiagnosticPlots(pred, test_data, train_data, qcd_hist, mask)
_,ax1 = p["gpr_mean"]
indp=model.covar_module.inducing_points
ind = test_transform.transform_x.iTransformData(indp).detach()

#ax1.scatter(ind[:,0], ind[:,1])
